In [1]:
import numpy as np
import librosa as lba
import soundfile as sf
import matplotlib.pyplot as plt
from random import random

In [2]:
stimuli_folder = '../stimuli/'

sr = 44100
tone_duration = 250

rise_duration = 10
rise_length = int(rise_duration * sr / 1000)

perc_duration = 240
perc_length = int(perc_duration * sr / 1000)

fade_duration = 10
fade_length = int(fade_duration * sr / 1000)

tones = {

    'B7': 3951.066,
    'A7': 3520,
    'Gs7': 3322.438,
    'Fs7': 2959.955,
    'F7': 2793.826,
    'Ds7': 2489.016,
    'D7': 2349.318,
    'C7': 2093.005,
    
    'G6': 1567.982,
    'F6': 1396.913,
    
    'B5': 987.7666,
    'A5': 880,
    'Gs5': 830.6094,
    'Fs5': 739.9888,
    'F5': 698.4565,
    'Ds5': 622.2540,
    'D5': 587.3295,
    'C5': 523.2511,
    
    'G4': 391.9954,
    'F4': 349.2282,
    
    'B3': 246.9417,
    'A3': 220,
    'Gs3': 207.6523,
    'Fs3': 184.9972,
    'F3': 174.6141,
    'Ds3': 155.5635,
    'D3': 146.8324,
    'C3': 130.8128
}

# Generate Tones

In [3]:
for pitch in tones:

    f = tones[pitch]
    
    # Generate fundamental frequency (co)sine wave
    tone = lba.tone(f, sr=sr, duration=tone_duration / 1000)
    
    # Add first three harmonics with slope of -6 db/half amplitude per octave
    for i in range(3):
        phase = random() * 2 * np.pi
        tone += lba.tone(f * (i + 2), sr=sr, duration=tone_duration / 1000, phi=phase) / (i + 2)
    
    # Rescale waveform to range [-1, 1] to prevent clipping
    tone /= np.abs(tone).max()

    # Apply exponential fade to create percussive envelope
    tone[-perc_length:] *= np.geomspace(1, .01, perc_length)
    # Apply short linear fade to ending so that amplitude fades to 0
    tone[-fade_length:] *= np.linspace(1, 0, fade_length)
    # Apply sharp linear rise to start of tone
    tone[:rise_length] *= np.linspace(0, 1, rise_length)
    
    # Save tone
    sf.write('../stimuli/tones/tone%s.wav' % pitch, tone, 44100)

## (Balance tones in Audacity now, then continue below)

# Generate Sequences

In [4]:
base_intervals = 4
gap_intervals = 2
mod_intervals = 1
ntones = base_intervals + gap_intervals + mod_intervals + 1

base_iois = np.array([400, 600])
offsets = [-.3, -.2, -.1, 0, .1, .2, .3]

click, _ = lba.load(stimuli_folder + 'tones/click-normed.wav', sr=sr)  # Note: Click must be same sample rate as tones

for pitch in tones:
    
    # Handle practice tones separately
    if pitch[-1] in ('4', '6'):
        continue
        
    # Load normalized tone of current octave
    tone, _ = lba.load(stimuli_folder + 'tones/tone%s-normed.wav' % pitch, sr=sr)

    for tempo in base_iois:

        for offset in offsets:

            # Create array of appropriate length to hold audio sequence
            ms_ioi = tempo * (1 + offset) / 1000  # Milisecond interval preceding probe tone
            tone_sequence = np.zeros(int(np.ceil(ntones * max(tempo / 1000, ms_ioi) * sr)), dtype=np.float32)
            click_sequence = tone_sequence.copy()

            # Insert tones at appropriate locations
            for i in range(ntones):

                # First several tones are spaced by the base IOI
                if i <= base_intervals:
                    start = i * tempo / 1000 * sr
                    start = int(np.ceil(start))

                    # Place a copy of the tone/click in the proper location
                    tone_end = start + tone.shape[0]
                    click_end = start + click.shape[0]
                    tone_sequence[start:tone_end] = tone
                    click_sequence[start:click_end] = click

                # Several silent intervals separate the entrainment sequence from the probe tone
                elif base_intervals < i <= base_intervals + gap_intervals:
                    continue

                # Final tone(s) is/are preceded by the modified IOI  
                else:
                    start = sr * (((base_intervals + gap_intervals) * tempo / 1000) + ((i - base_intervals - gap_intervals) * ms_ioi))
                    start = int(np.ceil(start))

                    # The tone sequence ends with a click; the click sequence ends with a tone
                    tone_end = start + tone.shape[0]
                    click_end = start + click.shape[0]
                    tone_sequence[start:click_end] = click
                    click_sequence[start:tone_end] = tone

            # Cut silence from the end of the sequence
            tone_sequence = np.trim_zeros(tone_sequence, 'b')
            click_sequence = np.trim_zeros(click_sequence, 'b')

            # Save sequences to WAV file
            sf.write(stimuli_folder + 'sequence_0_%s_%i_%i.wav' % (pitch, tempo, offset * 100), tone_sequence, sr)
            sf.write(stimuli_folder + 'sequence_1_%s_%i_%i.wav' % (pitch, tempo, offset * 100), click_sequence, sr)

# Generate Practice Sequences

In [5]:
base_intervals = 4
gap_intervals = 2
mod_intervals = 1
ntones = base_intervals + gap_intervals + mod_intervals + 1

base_iois = np.array([500])
offsets = [-.3, .3]

click, _ = lba.load(stimuli_folder + 'tones/click-normed.wav', sr=sr)  # Note: Click must be same sample rate as tones

for pitch in tones:
    
    # Only handle practice tones
    if pitch[-1] not in ('4', '6'):
        continue
        
    # Load normalized tone of current octave
    tone, _ = lba.load(stimuli_folder + 'tones/tone%s-normed.wav' % pitch, sr=sr)

    for tempo in base_iois:

        for offset in offsets:

            # Create array of appropriate length to hold audio sequence
            ms_ioi = tempo * (1 + offset) / 1000  # Milisecond interval preceding probe tone
            tone_sequence = np.zeros(int(np.ceil(ntones * max(tempo / 1000, ms_ioi) * sr)), dtype=np.float32)
            click_sequence = tone_sequence.copy()

            # Insert tones at appropriate locations
            for i in range(ntones):

                # First several tones are spaced by the base IOI
                if i <= base_intervals:
                    start = i * tempo / 1000 * sr
                    start = int(np.ceil(start))

                    # Place a copy of the tone/click in the proper location
                    tone_end = start + tone.shape[0]
                    click_end = start + click.shape[0]
                    tone_sequence[start:tone_end] = tone
                    click_sequence[start:click_end] = click

                # Several silent intervals separate the entrainment sequence from the probe tone
                elif base_intervals < i <= base_intervals + gap_intervals:
                    continue

                # Final tone(s) is/are preceded by the modified IOI  
                else:
                    start = sr * (((base_intervals + gap_intervals) * tempo / 1000) + ((i - base_intervals - gap_intervals) * ms_ioi))
                    start = int(np.ceil(start))

                    # The tone sequence ends with a click; the click sequence ends with a tone
                    tone_end = start + tone.shape[0]
                    click_end = start + click.shape[0]
                    tone_sequence[start:click_end] = click
                    click_sequence[start:tone_end] = tone

            # Cut silence from the end of the sequence
            tone_sequence = np.trim_zeros(tone_sequence, 'b')
            click_sequence = np.trim_zeros(click_sequence, 'b')

            # Save sequences to WAV file
            sf.write(stimuli_folder + 'sequence_0_%s_%i_%i.wav' % (pitch, tempo, offset * 100), tone_sequence, sr)
            sf.write(stimuli_folder + 'sequence_1_%s_%i_%i.wav' % (pitch, tempo, offset * 100), click_sequence, sr)